<a href="https://colab.research.google.com/github/yalperb/Audio-Calptioning/blob/main/ECG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d taejoongyoon/mitbit-arrhythmia-database

In [ ]:
! unzip mitbit-arrhythmia-database

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
beats= []
types= []

In [ ]:
for filename in os.listdir("mitbih_database/mitbih_database"):
	if filename.split(".")[1] == "csv":
		ecg = pd.read_csv("mitbih_database/mitbih_database/"+filename)
		file = open("mitbih_database/mitbih_database/"+filename.split(".")[0]+"annotations.txt", "r")
		for line in file.readlines()[1:]:
			index = int(line[12:21])
			if index < 200:
				beat = np.array(ecg[ecg.columns[1:]][0:index+200], dtype=int)
			elif index > len(ecg)-201:
				beat = np.array(ecg[ecg.columns[1:]][index-200:], dtype=int)
			else:
				beat = np.array(ecg[ecg.columns[1:]][index-200:index+200], dtype=int)
			beats.append(np.resize(beat, (100, 2)))
			types.append(line[26])
		file.close()

In [ ]:
beats = np.array(beats)
types = np.array(types)
print(types.shape)
print(beats.shape)

(112647,)
(112647, 100, 2)


In [ ]:
beats_filtered = []
types_filtered = []

In [ ]:
for j in ["N", "L", "R", "A", "V"]:
  k = 0
  for i in range(len(beats)):
    if types[i] == j :
      beats_filtered.append(beats[i])
      types_filtered.append(types[i])
      k += 1
      if k == 2500:
        break

In [ ]:
np.array(beats_filtered).shape

(12500, 100, 2)

In [ ]:
X = np.array(beats_filtered)
y = np.array(types_filtered)

In [ ]:
X.shape

(12500, 100, 2)

In [ ]:
from sklearn import preprocessing

In [ ]:
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.33, random_state=24)
print(X_train.shape, X_val_test.shape, y_train.shape, y_val_test.shape)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.50, random_state=24)
print(X_val.shape, X_test.shape, y_val.shape, y_test.shape)

(8375, 100, 2) (4125, 100, 2) (8375,) (4125,)
(2062, 100, 2) (2063, 100, 2) (2062,) (2063,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
model = models.Sequential()
model.add(layers.Conv1D(64, (3), activation='relu', input_shape=(100, 2)))
model.add(layers.MaxPooling1D((3)))
model.add(layers.Conv1D(64, (5), activation='relu'))
model.add(layers.MaxPooling1D((3)))
model.add(layers.Conv1D(128, (5), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
131/131 [==============================] - 6s 34ms/step - loss: 3.7900 - accuracy: 0.4765 - val_loss: 0.8676 - val_accuracy: 0.6610
Epoch 2/100
131/131 [==============================] - 4s 31ms/step - loss: 0.8764 - accuracy: 0.6558 - val_loss: 0.7044 - val_accuracy: 0.7260
Epoch 3/100
131/131 [==============================] - 3s 24ms/step - loss: 0.7700 - accuracy: 0.6939 - val_loss: 0.7298 - val_accuracy: 0.7313
Epoch 4/100
131/131 [==============================] - 3s 24ms/step - loss: 0.6980 - accuracy: 0.7235 - val_loss: 0.6333 - val_accuracy: 0.7386
Epoch 5/100
131/131 [==============================] - 3s 24ms/step - loss: 0.6453 - accuracy: 0.7339 - val_loss: 0.7642 - val_accuracy: 0.7163
Epoch 6/100
131/131 [==============================] - 3s 24ms/step - loss: 0.6110 - accuracy: 0.7531 - val_loss: 0.6004 - val_accuracy: 0.7793
Epoch 7/100
131/131 [==============================] - 3s 24ms/step - loss: 0.5647 - accuracy: 0.7693 - val_loss: 0.5666 - val_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=0)
print(test_acc)

0.9069316387176514
